In [ ]:
import os
import time
import logging
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy import concatenate_videoclips

# Set up logging
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO,  # Adjust log level here (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    handlers=[logging.StreamHandler()]  # Output to console
)

# Suppress the verbose logging from moviepy/ffmpeg
logging.getLogger("moviepy").setLevel(logging.ERROR)
logging.getLogger("ffmpeg").setLevel(logging.ERROR)

# Google API configuration
SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]
CLIENT_SECRETS_FILE = "client_secret.json"
TOKEN_FILE = "token.json"
YOUTUBE_MAX_DURATION = 43200  # 12 hours in seconds

MERGE_TIMEOUT = 2400  # 40 minutes
CONVERT_TIMEOUT = 7200  # 2 hours
MAX_VIDEO_DURATION = 21600  # 6 hours in seconds


def get_authenticated_service():
    """
    Authenticate with YouTube API and return a service object.
    """
    creds = None

    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                logging.info("🔄 Token expired. Attempting to refresh...")
                creds.refresh(Request())
                logging.info("✅ Token refreshed successfully!")
            except Exception as e:
                logging.error(f"⚠️ Token refresh failed: {e}\n🌐 Opening browser for re-authentication...")
                creds = None  # Force re-authentication

        if creds is None:
            logging.info("🌐 Opening browser for authentication...")
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        with open(TOKEN_FILE, "w") as token_file:
            token_file.write(creds.to_json())

    return build("youtube", "v3", credentials=creds)

def get_video_duration(video_path):
    """
    Returns the duration of a video in seconds.
    """
    try:
        video_clip = VideoFileClip(video_path)
        duration = video_clip.duration
        video_clip.close()
        return duration
    except Exception as e:
        logging.warning(f"⚠️ Warning: Could not read duration for {video_path}. Error: {e}")
        return None

def convert_videos(folder_path, video_files):
    """
    Converts videos to a uniform format (same codec, resolution, frame rate, etc.).
    Skips videos with no valid duration.
    """
    converted_folder = os.path.join(folder_path, "converted_videos")
    os.makedirs(converted_folder, exist_ok=True)
    converted_files = []

    # Filter out videos with no valid duration
    video_files = [f for f in video_files if get_video_duration(f) is not None]

    if not video_files:
        logging.warning(f"⚠️ No valid videos to convert in {folder_path}. Skipping...")
        return []

    logging.info(f"🔄 Starting video conversion... (Timeout: {CONVERT_TIMEOUT // 3600} hours)")

    start_time = time.time()
    for video in video_files:
        converted_video = os.path.join(converted_folder, os.path.basename(video))

        try:
            video_clip = VideoFileClip(video)
            video_clip = video_clip.resize(height=720)  # Resize to 720p
            video_clip.write_videofile(converted_video, codec="libx264", audio_codec="aac", threads=4)
            video_clip.close()
        except Exception as e:
            logging.error(f"❌ Error during conversion of {video}: {e}")
            continue

        converted_files.append(converted_video)

        elapsed_time = time.time() - start_time
        if elapsed_time > CONVERT_TIMEOUT:
            logging.warning("⏳ Total conversion exceeded allowed time. Aborting conversion...")
            return []

    logging.info("✅ Video conversion complete!")
    return converted_files

def try_merge_videos(folder_path, video_files, expected_duration, merge_attempt):
    """
    Merges videos using MoviePy.
    """
    merged_video_path = os.path.join(folder_path, "merged_video.mp4")

    # Filter out videos with no valid duration
    video_files = [f for f in video_files if get_video_duration(f) is not None]

    if not video_files:
        logging.warning(f"⚠️ No valid videos to merge in {folder_path}. Skipping...")
        return None

    logging.info(f"🔗 Attempt {merge_attempt}: Merging videos... (Timeout: {MERGE_TIMEOUT // 60} min)")

    try:
        video_clips = [VideoFileClip(f) for f in video_files]
        final_clip = concatenate_videoclips(video_clips, method="compose")
        final_clip.write_videofile(merged_video_path, codec="libx264", audio_codec="aac", threads=4)
        final_clip.close()

        merged_duration = get_video_duration(merged_video_path)
        if merged_duration and abs(merged_duration - expected_duration) <= 5:
            logging.info(f"✅ Merge successful! Duration: {merged_duration:.2f} sec")
            return merged_video_path
        else:
            logging.error(f"❌ Merge failed. Merged duration: {merged_duration:.2f} sec, Expected: {expected_duration:.2f} sec")
            os.remove(merged_video_path)
            return None
    except Exception as e:
        logging.error(f"❌ Error during merge: {e}")
        return None

def merge_videos_and_create_timestamps(folder_path, video_files, part_number=None):
    """
    Merges videos, creates timestamps, and returns the final video path, description, and total duration.
    """
    logging.info(f"📂 Checking folder: {folder_path}")

    if not os.path.exists(folder_path):
        logging.error(f"❌ Error: The folder path '{folder_path}' does not exist.")
        return None, None, None

    # Skip corrupted videos
    video_files = [f for f in video_files if get_video_duration(f) is not None]

    if not video_files:
        logging.warning(f"⚠️ No valid videos found in {folder_path}. Skipping folder...")
        return None, None, None

    total_duration = sum(get_video_duration(f) or 0 for f in video_files)

    # Split videos if total duration > 6 hours
    if total_duration > MAX_VIDEO_DURATION:
        logging.warning(f"⚠️ Video duration too long ({total_duration} sec). Splitting into two parts...")
        mid_index = len(video_files) // 2
        first_half = video_files[:mid_index]
        second_half = video_files[mid_index:]

        merged_video_path_1 = try_merge_videos(folder_path, first_half, sum(get_video_duration(f) or 0 for f in first_half), merge_attempt=1)
        merged_video_path_2 = try_merge_videos(folder_path, second_half, sum(get_video_duration(f) or 0 for f in second_half), merge_attempt=2)

        if merged_video_path_1 and merged_video_path_2:
            return [merged_video_path_1, merged_video_path_2], "\n".join(os.path.basename(f) for f in video_files), total_duration
        else:
            return None, None, None

    # Try merging normally if duration is within limit
    merged_video_path = try_merge_videos(folder_path, video_files, total_duration, merge_attempt=1)
    if merged_video_path:
        return merged_video_path, "\n".join(os.path.basename(f) for f in video_files), total_duration

    logging.info("🔄 Converting videos due to merge failure...")
    converted_files = convert_videos(folder_path, video_files)

    if not converted_files:
        logging.error(f"❌ Conversion failed for {folder_path}. Skipping upload.")
        return None, None, None

    return try_merge_videos(folder_path, converted_files, total_duration, merge_attempt=2), "\n".join(os.path.basename(f) for f in converted_files), total_duration


def upload_to_youtube(video_path, title, description, folder_path):
    """
    Uploads a video to YouTube and saves the video URL in 'youtube link.txt'.
    """
    logging.info(f"📤 Uploading: {title}...")
    youtube = get_authenticated_service()
    request = youtube.videos().insert(
        part="snippet,status",
        body={"snippet": {"title": title, "description": description, "categoryId": "27"}, "status": {"privacyStatus": "unlisted"}},
        media_body=MediaFileUpload(video_path, chunksize=-1, resumable=True)
    )

    _, response = request.next_chunk()
    video_url = f"https://www.youtube.com/watch?v={response['id']}"
    with open(os.path.join(folder_path, "youtube_link.txt"), "w") as f:
        f.write(video_url + "\n")

    logging.info(f"✅ Upload complete! Video URL: {video_url}")
    return video_url

if __name__ == "__main__":
    parent_folder = input("📂 Enter the parent folder containing all video subfolders: ").strip()

    for subfolder in sorted(os.listdir(parent_folder)):
        subfolder_path = os.path.join(parent_folder, subfolder)

        if os.path.isdir(subfolder_path):
            logging.info(f"🚀 Processing folder: {subfolder_path}")

            video_files = sorted([os.path.join(subfolder_path, f) for f in os.listdir(subfolder_path) if f.endswith('.mp4')])
            merged_video_path, description, _ = merge_videos_and_create_timestamps(subfolder_path, video_files)
            if merged_video_path:
                upload_to_youtube(merged_video_path, subfolder, description, subfolder_path)

    logging.info("✅ All subfolders processed!")


2025-02-27 10:19:09,797 - INFO - 🚀 Processing folder: H:\Projects Control (PC)\10 Backup\05 Tutorials\Adobe\Substance Designer\Lynda\Substance Designer Essential Training
2025-02-27 10:19:09,801 - INFO - 📂 Checking folder: H:\Projects Control (PC)\10 Backup\05 Tutorials\Adobe\Substance Designer\Lynda\Substance Designer Essential Training


{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'mp42isom', 'creation_time': '2017-08-22T17:09:18.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': 'eng', 'default': True, 'size': [1280, 720], 'bitrate': 554, 'fps': 15.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'handler_name': 'Telestream Inc. Dib Library - ReleaseVantage 2014.59.138974', 'vendor_id': 'TELE', 'encoder': 'Apple ProRes 422'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': 160, 'metadata': {'Metadata': '', 'handler_name': 'Telestream Inc. Dib Library - ReleaseVantage 2014.59.138974', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 167.13, 'bitrate': 720, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'vi

2025-02-27 10:22:38,568 - INFO - 🔗 Attempt 1: Merging videos... (Timeout: 40 min)


{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'mp42isom', 'creation_time': '2017-08-22T17:09:18.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': 'eng', 'default': True, 'size': [1280, 720], 'bitrate': 554, 'fps': 15.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'handler_name': 'Telestream Inc. Dib Library - ReleaseVantage 2014.59.138974', 'vendor_id': 'TELE', 'encoder': 'Apple ProRes 422'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': 160, 'metadata': {'Metadata': '', 'handler_name': 'Telestream Inc. Dib Library - ReleaseVantage 2014.59.138974', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 167.13, 'bitrate': 720, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'vi

MoviePy - Done.
MoviePy - Writing video H:\Projects Control (PC)\10 Backup\05 Tutorials\Adobe\Substance Designer\Lynda\Substance Designer Essential Training\merged_video.mp4



frame_index:   2%|▏         | 6165/263985 [15:28<10:56:29,  6.55it/s, now=None] 